In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from dynamic_routing_analysis import decoding_utils
from dynamic_routing_analysis import plot_utils
# from dynamic_routing_analysis import spike_utils

import os
import pingouin as pg

import matplotlib
import matplotlib.font_manager as fm

matplotlib.rcParams['font.size'] = 8
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
fm.FontProperties().set_family('arial')

%load_ext autoreload
%autoreload 2
# %matplotlib inline
%matplotlib widget

In [ ]:
# results_path=r's3://aind-scratch-data/dynamic-routing/decoding/results/binsize_0.5_sec_0/'
# results_path=r's3://aind-scratch-data/dynamic-routing/decoding/results/binsize_1.5_sec_0/'
# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/rewarded_modality_1.5sec_0/" #1.5 sec - typical context decoding
# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_response_1.5sec_0/" #1.5 sec - context appropriate response
# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_response_1_5sec_incl_hits_0/" # 1.5 sec - context appropriate response including hits

# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_response_1_5sec_control_0/" #decode rewarded_modality with same trials as context_appropriate_response (is_target=True)

# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_for_response_0_5sec_0/"
# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_for_response_0_2sec_0/"

# results_path = "s3://aind-scratch-data/dynamic-routing/decoding/results/context_appropriate_for_response_control_0_5sec_0/"

# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/decode_context_w_shifts_min_10_units_500ms_0/"

# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/time_mod_keep_units_consistent_0/"

# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_blockwise_crossval_w_repeats_and_linear_shift_20251215_214910/" #blockwise crossval with repeats and linear shift (example)
# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_5fold_crossval_w_repeats_and_linear_shift_20251215_220056/" #5fold crossval with repeats and linear shift (example)
# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_blockwise_crossval_w_repeats_and_linear_shift_0/"
# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_blockwise_crossval_w_repeats_and_linear_shift_strict_unit_drift_0/"
# results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_blockwise_crossval_w_repeats_and_linear_shift_strict_drift_20_units_0/"
results_path = r"s3://aind-scratch-data/dynamic-routing/decoding/results/test_blockwise_crossval_w_repeats_and_linear_shift_strict_unit_drift_0/"

session_table_path=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\CO decoding results\session_table_v0.272.parquet"
session_table=pl.read_parquet(session_table_path)

dr_session_list=(
    session_table.filter(
    pl.col('project')=="DynamicRouting",
    pl.col('is_production'),
    pl.col('is_annotated'),
    pl.col('issues')==[],
    pl.col('is_good_behavior').eq(True),
    pl.col('is_engaged').eq(True),
    )['session_id'].to_list()
    )

## Structure-averaged results (across sessions)

In [ ]:
results_df = decoding_utils.load_structure_average_decoder_accuracy(
    results_path, 
    session_list=dr_session_list, 
    combine_multi_probe_rec=True, 
    exclude_redundant_structures=True, 
    exclude_general_structures=True
    )

In [ ]:
results_pd=results_df.to_pandas()
results_pd

In [ ]:
# savepath=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\CO decoding results\time_mod_keep_units_consistent_0_2025-06-15"
# results_pd.to_parquet(os.path.join(savepath,"structure_wise_decoding_accuracy.parquet"))

In [ ]:
# barplot of decoding accuracy by structure, separate plot for each unit subsample size
plot_column='true' #true, null, diff
min_n_sessions=3
for uu in results_pd['unit_subsample_size'].unique():
    fig,ax=plt.subplots(1,1,figsize=(12,4))
    structure_labels=[]
    if np.isnan(uu):
        plot_data=results_pd.query('unit_subsample_size.isna() and num_sessions>=@min_n_sessions').sort_values(by=f'mean_{plot_column}',ascending=False)
        plot_data.plot.bar(ax=ax,x='structure',y=[f'mean_{plot_column}'],yerr=f'sem_{plot_column}',rot=90)
        title_str='All units'
        for i in range(plot_data.shape[0]):
            structure_labels.append(plot_data['structure'].iloc[i]+' ('+str(int(plot_data['num_sessions'].iloc[i]))+')')
    else:
        plot_data=results_pd.query('unit_subsample_size==@uu and num_sessions>=@min_n_sessions').sort_values(by=f'mean_{plot_column}',ascending=False)
        plot_data.plot.bar(ax=ax,x='structure',y=[f'mean_{plot_column}'],yerr=f'sem_{plot_column}',rot=90)
        title_str=f'{int(uu)} units'
        for i in range(plot_data.shape[0]):
            structure_labels.append(plot_data['structure'].iloc[i]+' ('+str(int(plot_data['num_sessions'].iloc[i]))+')')

    ax.set_ylabel('decoding accuracy above null')
    ax.set_title(title_str)
    ax.set_xticklabels(structure_labels,rotation=90,ha='center')
    
    if plot_column!='diff':
        ax.axhline(0.5, color='red', linestyle='--')
    fig.tight_layout()


In [ ]:
#plot on CCF

sel_n_units=10
sel_bin_center=-0.25
min_n_sessions=3

if sel_n_units == 'all':
    query_string='unit_subsample_size.isna() and bin_center==@sel_bin_center and num_sessions>=@min_n_sessions'
else:
    query_string='unit_subsample_size==@sel_n_units and bin_center==@sel_bin_center and num_sessions>=@min_n_sessions'

chart,x = plot_utils.plot_brain_heatmap(
    regions=results_pd.query(query_string)['structure'],
    values=results_pd.query(query_string)['mean_diff'],
    # values=np.ones(results_pd.query(query_string).shape[0])*0.027,
    # sagittal_planes=(-250,-500,-750,-1000,-1500,-2000,-3000),
    sagittal_planes=(-300,-1000,-2000,),
    # sagittal_planes=(-1500),
    cmap='Reds',
    # cmap='Greys',
    clevels=(0.02,0.14),
    # clevels=(0.55,0.8),
    # labels=True,
    # labels_on_areas=True,
    interactive=True,
    annotation_params={
        'fontsize':6,
    }
)

# plt.savefig(os.path.join(savepath,"decoding_realbrain_heatmap_raw_acc.png"),dpi=600,bbox_inches='tight')

In [ ]:
# show interactive CCF plot
chart.show()

## Plot decoding accuracy vs. n units used for decoding
### (requires results with multiple values of unit_subsample_size )

In [ ]:
#accuracy vs. n units for a subset of structures

sel_structures=['SCm','MRN','MOs','FRP','ACAd','CP','PL','ORBl','CA1','VISp']

for ss in sel_structures:
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    plot_data=results_pd.query('structure==@ss').sort_values(by='mean_diff',ascending=True)

    ax.errorbar(plot_data.query('~unit_subsample_size.isna()')['unit_subsample_size'],
                plot_data.query('~unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('~unit_subsample_size.isna()')['sem_diff'],marker='o',color='k')
    ax.errorbar(np.nanmax(plot_data['unit_subsample_size'])+10,
                plot_data.query('unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('unit_subsample_size.isna()')['sem_diff'],marker='o',color='k')
    
    for uu in plot_data['unit_subsample_size'].unique():
        if np.isnan(uu):
            xcoord=np.nanmax(plot_data['unit_subsample_size'])+10
            ycoord=plot_data.query('unit_subsample_size.isna()')[['mean_diff','sem_diff']].sum(axis=1).values[0]
            temp_str=plot_data.query('unit_subsample_size.isna()')['num_sessions'].values[0]
        else:
            xcoord=uu
            ycoord=plot_data.query('unit_subsample_size==@uu')[['mean_diff','sem_diff']].sum(axis=1).values[0]
            temp_str=plot_data.query('unit_subsample_size==@uu')['num_sessions'].values[0]

        ax.text(xcoord,ycoord,temp_str,ha='center',va='bottom')

    ax.set_ylabel('decoding accuracy above null')
    ax.set_xlabel('unit subsample size')
    ax.set_xticks(plot_data['unit_subsample_size'].unique().tolist()+[np.nanmax(plot_data['unit_subsample_size'])+10])
    ax.set_xticklabels(plot_data['unit_subsample_size'].unique().astype(int).tolist()+['all_units'])
    ax.set_title(ss)
    ax.set_ylim([0,0.2])
    fig.tight_layout()

#plot all on same figure
fig,ax=plt.subplots(1,1,figsize=(5,4))
for ss in sel_structures:
    plot_data=results_pd.query('structure==@ss').sort_values(by='mean_diff',ascending=True)

    line=ax.errorbar(plot_data.query('~unit_subsample_size.isna()')['unit_subsample_size'],
                plot_data.query('~unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('~unit_subsample_size.isna()')['sem_diff'],marker='o',label=ss)
    ax.errorbar(np.nanmax(plot_data['unit_subsample_size'])+10,
                plot_data.query('unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('unit_subsample_size.isna()')['sem_diff'],marker='o',
                color=line[0].get_color())
    
ax.legend()
ax.set_ylabel('decoding accuracy above null')
ax.set_xlabel('unit subsample size')
ax.set_xticks(plot_data['unit_subsample_size'].unique().tolist()+[np.nanmax(plot_data['unit_subsample_size'])+10])
ax.set_xticklabels(plot_data['unit_subsample_size'].unique().astype(int).tolist()+['all_units'])
ax.set_ylim([0,0.2])
fig.tight_layout()

## Session-wise results

In [ ]:
results_session_df=decoding_utils.load_session_wise_decoder_accuracy(results_path, dr_session_list, session_table,)

In [ ]:
results_session_pd=results_session_df.to_pandas()


In [ ]:
# save table output
# results_session_pd.to_parquet(os.path.join(savepath,"session_wise_decoding_accuracy.parquet"),index=False)

In [ ]:
results_session_pd

## Significance testing

In [ ]:
#example significance test on one structure
sel_structure='GPe'

values=results_session_pd.query('structure==@sel_structure')['mean_true'].values-0.5

stats.ttest_1samp(values, 0)

In [ ]:
#example significance test between structures
s1='MOs'
s2='SSp'

values1=results_session_pd.query('structure==@s1')['mean_true'].values-0.5
values2=results_session_pd.query('structure==@s2')['mean_true'].values-0.5

stats.mannwhitneyu(values1,values2)

In [ ]:
#calculate statistical difference from chance (0.5) for each structure
#save p value and mean_true values

structure_decoding_significance={
    'structure': [],
    'p_value': [],
    'statistic': [],
    'mean_true': [],
    'sem_true': [],
}

min_n_sessions=3
for sel_structure in results_session_pd['structure'].unique():
    structure_table=results_session_pd.query('structure==@sel_structure')
    if structure_table.shape[0]<min_n_sessions:
        continue
    values=structure_table['mean_true'].values-0.5
    stat,p_value=stats.ttest_1samp(values, 0)
    structure_decoding_significance['structure'].append(sel_structure)
    structure_decoding_significance['p_value'].append(p_value)
    structure_decoding_significance['statistic'].append(stat)
    structure_decoding_significance['mean_true'].append(np.mean(structure_table['mean_true'].values))
    structure_decoding_significance['sem_true'].append(stats.sem(structure_table['mean_true'].values))

#adjust pvals with BH correction
pvals=np.array(structure_decoding_significance['p_value'])
rej, pvals_corrected = pg.multicomp(pvals, alpha=0.05, method='fdr_bh')

structure_decoding_significance['p_value_corrected']=pvals_corrected

structure_decoding_significance_df=pd.DataFrame(structure_decoding_significance)
structure_decoding_significance_df

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,4))
structure_labels=[]
plot_data=structure_decoding_significance_df.sort_values(by='mean_true',ascending=False)

significance=plot_data['p_value_corrected']<0.05

#make bars red if significant, grey if not significant
bars=plot_data.plot.bar(ax=ax,x='structure',y=['mean_true'],yerr='sem_true',rot=90)

# Change face color based on significance
for i, (bar, is_sig) in enumerate(zip(bars.patches, significance)):
    if not is_sig:
        bar.set_facecolor('grey')

for i in range(plot_data.shape[0]):
    structure_labels.append(plot_data['structure'].iloc[i])

ax.axhline(0.5, color='red', linestyle='--', alpha=0.5)

ax.set_ylim([0.45,0.85])
ax.set_ylabel('decoding accuracy')
ax.set_xlabel('structure')
ax.set_title('Decoding accuracy by structure')
ax.set_xticklabels(structure_labels,rotation=90,ha='center')
fig.tight_layout()


### ANOVA

In [ ]:
#anova on structure, n_passing_blocks 
pg.anova(results_session_pd.query('unit_subsample_size==10'), dv='mean_true', between=['structure','n_passing_blocks'])

In [ ]:
# ###(not currently working!)
# #ancova on structure, mean_cross_modal_dprime 
# pg.ancova(results_session_pd.query('unit_subsample_size==10'), dv='mean_true', between='structure', covar='n_passing_blocks')

In [ ]:
#welch anova
pg.welch_anova(results_session_pd.query('unit_subsample_size==10'), dv='mean_diff', between='structure')

In [ ]:
#post-hoc Tukey tests between structures
pairwise_tukey_pd=pg.pairwise_tukey(results_session_pd.query('unit_subsample_size==10'), dv='mean_true', between='structure')
#rename column to allow querying
pairwise_tukey_pd['p_tukey'] = pairwise_tukey_pd['p-tukey']

pairwise_tukey_pd.query('p_tukey<0.05').sort_values(by='mean(A)',ascending=False)

In [ ]:
#alternative post-hoc test
pairwise_gameshowell_pd=pg.pairwise_gameshowell(results_session_pd.query('unit_subsample_size==10'), dv='mean_diff', between='structure')
pairwise_gameshowell_pd

In [ ]:
#make difference/comparison matrices
unique_structures = pairwise_gameshowell_pd['A'].unique()

n_struct = len(unique_structures)
pair_idx = pd.MultiIndex.from_product([unique_structures, unique_structures], names=['A', 'B'])

pairwise_indexed = pairwise_gameshowell_pd.set_index(['A', 'B']).reindex(pair_idx)

reshaped_pval = pairwise_indexed['pval'].to_numpy().reshape(n_struct, n_struct).copy()
reshaped_meanA = pairwise_indexed['mean(A)'].to_numpy().reshape(n_struct, n_struct).copy()
reshaped_mean_diff = pairwise_indexed['diff'].to_numpy().reshape(n_struct, n_struct).copy()

np.fill_diagonal(reshaped_pval, np.nan)
np.fill_diagonal(reshaped_meanA, np.nan)
np.fill_diagonal(reshaped_mean_diff, np.nan)

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(6,6))
ax.imshow(reshaped_mean_diff, cmap='Reds',)# vmin=0, vmax=1)
fig.colorbar(ax.images[0], ax=ax, orientation='vertical', label='mean difference in decoding accuracy')

## Box plot of decoding accuracy for one structure across sessions

In [ ]:
# barplot for one structure across sessions

matplotlib.rcParams['font.size'] = 10

#plot boxplot of chosen variable for list of structures
structure_query='ORBm|ORBvl|ORBl'
# structure_list=['ORBm','ORBvl','ORBl']
structure_list=['ORB']

sel_column='context'

fig,ax=plt.subplots(1,1,figsize=(1.5,3))

for ss,structure_query in enumerate(structure_list):

    session_wise_results=results_session_pd.query('structure.str.contains(@structure_query) and session_id in @dr_session_list')

    data = session_wise_results['mean_true'].values
    ax.boxplot(data,positions=[ss], widths=0.6, showfliers=False, medianprops=dict(color='black'))

    #jittered scatter plot of individual data points
    ax.scatter(np.random.normal(ss, 0.1, size=data.shape), data, alpha=0.2, color='k')

#remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xticks(np.arange(len(structure_list)))
ax.set_xticklabels(structure_list,rotation=45,ha='right')
ax.set_yticks(np.arange(0.5,0.91,0.1))
ax.set_ylabel('context decoding accuracy')
ax.set_ylim(0.5,0.9)
fig.tight_layout()

In [ ]:

# figsavepath=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\quick figures\2025-10-27-muscimol_sfn_poster"
# fig.savefig(os.path.join(figsavepath,'context_decoding_accuracy_ORB_combined.png'),dpi=300)

## Plot session-wise results sorted by unit_subsample_size

In [ ]:
sessions_with_enough_units = (
    results_session_df
    .group_by('session_id','structure','n_passing_blocks')
    .agg(
        pl.col('unit_subsample_size','mean_true', 'median_null','mean_diff'),
    )
)

In [ ]:
max_n_units=10
sel_structures=['SCm','MRN','MOs','FRP','ACAd','CP','PL','ORBl','CA1','VISp']

fig,ax=plt.subplots(1,1,figsize=(5,4))
for sel_structure in sel_structures:

    sessions_with_enough_units_pd=sessions_with_enough_units.filter(
        pl.col('structure')==sel_structure,
        pl.col('unit_subsample_size').list.contains(max_n_units),
        pl.col('n_passing_blocks').ge(4)
    ).to_pandas()

    plot_data=sessions_with_enough_units_pd


    xvect=np.arange(len(plot_data['unit_subsample_size'].iloc[0]))
    xlabels=plot_data['unit_subsample_size'].iloc[0]

    line=ax.errorbar(xvect[~np.isnan(xlabels)],np.mean(np.vstack(plot_data['mean_diff'].values),axis=0)[~np.isnan(xlabels)],
                    yerr=stats.sem(np.vstack(plot_data['mean_diff'].values),axis=0)[~np.isnan(xlabels)],marker='o',
                    label=sel_structure+f' ({len(plot_data)})')

    #plot nan value at the end of the plot
    ax.errorbar(xvect[-1]+1,np.mean(np.vstack(plot_data['mean_diff'].values),axis=0)[np.isnan(xlabels)],
                yerr=stats.sem(np.vstack(plot_data['mean_diff'].values),axis=0)[np.isnan(xlabels)],marker='o',color=line[0].get_color())
    
ax.legend()
ax.set_ylabel('decoding accuracy above null')
ax.set_xlabel('unit subsample size')
ax.set_xticks(xvect+1)
ax.set_xticklabels(list(xlabels[1:])+['all_units'])
ax.set_ylim([0,0.2])
fig.tight_layout()



In [ ]:
#accuracy vs. n units for a subset of structures

sel_structures=['SCm','MRN','MOs','FRP','ACAd','CP','PL','ORBl','CA1','VISp']

for ss in sel_structures:
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    plot_data=results_pd.query('structure==@ss').sort_values(by='mean_diff',ascending=True)

    ax.errorbar(plot_data.query('~unit_subsample_size.isna()')['unit_subsample_size'],
                plot_data.query('~unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('~unit_subsample_size.isna()')['sem_diff'],marker='o',color='k')
    ax.errorbar(np.nanmax(plot_data['unit_subsample_size'])+10,
                plot_data.query('unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('unit_subsample_size.isna()')['sem_diff'],marker='o',color='k')
    
    for uu in plot_data['unit_subsample_size'].unique():
        if np.isnan(uu):
            xcoord=np.nanmax(plot_data['unit_subsample_size'])+10
            ycoord=plot_data.query('unit_subsample_size.isna()')[['mean_diff','sem_diff']].sum(axis=1).values[0]
            temp_str=plot_data.query('unit_subsample_size.isna()')['num_sessions'].values[0]
        else:
            xcoord=uu
            ycoord=plot_data.query('unit_subsample_size==@uu')[['mean_diff','sem_diff']].sum(axis=1).values[0]
            temp_str=plot_data.query('unit_subsample_size==@uu')['num_sessions'].values[0]

        ax.text(xcoord,ycoord,temp_str,ha='center',va='bottom')

    ax.set_ylabel('decoding accuracy above null')
    ax.set_xlabel('unit subsample size')
    ax.set_xticks(plot_data['unit_subsample_size'].unique().tolist()+[np.nanmax(plot_data['unit_subsample_size'])+10])
    ax.set_xticklabels(plot_data['unit_subsample_size'].unique().astype(int).tolist()+['all_units'])
    ax.set_title(ss)
    ax.set_ylim([0,0.2])
    fig.tight_layout()

#plot all on same figure
fig,ax=plt.subplots(1,1,figsize=(5,4))
for ss in sel_structures:
    plot_data=results_pd.query('structure==@ss').sort_values(by='mean_diff',ascending=True)

    line=ax.errorbar(plot_data.query('~unit_subsample_size.isna()')['unit_subsample_size'],
                plot_data.query('~unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('~unit_subsample_size.isna()')['sem_diff'],marker='o',label=ss)
    ax.errorbar(np.nanmax(plot_data['unit_subsample_size'])+10,
                plot_data.query('unit_subsample_size.isna()')['mean_diff'],
                yerr=plot_data.query('unit_subsample_size.isna()')['sem_diff'],marker='o',
                color=line[0].get_color())
    
ax.legend()
ax.set_ylabel('decoding accuracy above null')
ax.set_xlabel('unit subsample size')
ax.set_xticks(plot_data['unit_subsample_size'].unique().tolist()+[np.nanmax(plot_data['unit_subsample_size'])+10])
ax.set_xticklabels(plot_data['unit_subsample_size'].unique().astype(int).tolist()+['all_units'])
ax.set_ylim([0,0.2])
fig.tight_layout()

## Plot results relative to average CCF coordinates

#### should load results by probe insertion, not combine them

In [ ]:
session_structure_ccf_coords_df = decoding_utils.get_average_session_structure_ccf_coords(results_session_df,all_units_table_path=None)

results_session_with_ccf_pd=pd.merge(
    results_session_pd,
    session_structure_ccf_coords_df,
    on=['session_id','structure'],
    how='left'
)

In [ ]:
results_session_with_ccf_pd

In [ ]:
from allensdk.core.reference_space_cache import ReferenceSpaceCache
from pathlib import Path

output_dir = r"D:\ccf_cache"
reference_space_key = "annotation/ccf_2022"
resolution = 10
rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest=Path(output_dir) / 'manifest.json')

# get structure tree
# ID 1 is the adult mouse structure graph
tree = rspc.get_structure_tree(structure_graph_id=1) 

# get id_acronym map
id_acronym_map = tree.get_id_acronym_map()

# make reference space object
rsp = rspc.get_reference_space()

In [ ]:
# area_mask

In [ ]:
# structure_list=['ORBl','ORBm','ORBvl','FRP','PL']
structure_list=['MOs']
# dprime_threshold=1.0

structure_id_list=[id_acronym_map[structure_list[0]]]

if len(structure_list)>1:
    for sel_structure in structure_list[1:]:
        structure_id_list.append(id_acronym_map[sel_structure])

area_mask=rsp.make_structure_mask(structure_id_list)

In [ ]:
# #get separate area masks

area_masks={}

for ss in structure_list:
    structure_id_list=[id_acronym_map[ss]]
    area_masks[ss]=rsp.make_structure_mask(structure_id_list)


In [ ]:
combined_area_mask=area_masks[structure_list[0]]
for ii,ss in enumerate(structure_list[1:]):
    combined_area_mask+=(area_masks[ss]*(ii+1))

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.imshow(np.max(combined_area_mask[:,:,:],axis=0),cmap='magma',alpha=0.5)

In [ ]:
sel_structure='MOs'
sel_unit_subsample_size=10
plot_data=results_session_with_ccf_pd.query('structure==@sel_structure and n_passing_blocks>=5 and unit_subsample_size==@sel_unit_subsample_size')
# plot_data=results_session_with_ccf_pd.query(
#     '(structure=="ORBl" or structure=="ORBm" or structure=="ORBvl" or structure=="FRP" or structure=="PL") and \
#     n_passing_blocks>=5 and unit_subsample_size==@sel_unit_subsample_size')

#plot ccf_ap vs. ccf_ml vs. decoding accuracy

fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.imshow(np.max(area_mask[:,:,:],axis=1),cmap='gray',alpha=0.5)
# ax.imshow(np.max(combined_area_mask[:,:,:],axis=1),cmap='magma',alpha=0.5)

ax.scatter((plot_data['ccf_ml']/10),plot_data['ccf_ap']/10,c=plot_data['mean_diff'],s=20,cmap='Reds',vmin=0,vmax=0.2)

ax.set_ylabel('CCF AP')
ax.set_xlabel('CCF ML')

#set xlim and ylim to be around the area mask
ax.set_xlim([np.nanmin(plot_data['ccf_ml']/10)-100,np.nanmax(plot_data['ccf_ml']/10)+100])
ax.set_ylim([np.nanmin(plot_data['ccf_ap']/10)-100,np.nanmax(plot_data['ccf_ap']/10)+100])

ax.invert_yaxis()

#make colorbar
cbar = plt.colorbar(ax.collections[0], ax=ax, orientation='vertical')
cbar.set_label('decoding accuracy above null', rotation=90, labelpad=15)

ax.set_title(f'{sel_structure} {sel_unit_subsample_size} units; horizontal view')

# fig.tight_layout()

In [ ]:
#plot ccf_dv vs. ccf_ml vs. decoding accuracy

fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.imshow(np.max(area_mask[:,:,:],axis=0),cmap='gray',alpha=0.5)
# ax.imshow(np.max(combined_area_mask[:,:,:],axis=0),cmap='magma',alpha=0.5)

ax.scatter((plot_data['ccf_ml']/10),plot_data['ccf_dv']/10,c=plot_data['mean_diff'],s=20,cmap='Reds',vmin=0,vmax=0.2)

ax.set_ylabel('CCF DV')
ax.set_xlabel('CCF ML')

#set xlim and ylim to be around the area mask
ax.set_xlim([np.nanmin(plot_data['ccf_ml']/10)-100,np.nanmax(plot_data['ccf_ml']/10)+100])
ax.set_ylim([np.nanmin(plot_data['ccf_dv']/10)-100,np.nanmax(plot_data['ccf_dv']/10)+100])

ax.invert_yaxis()

#make colorbar
cbar = plt.colorbar(ax.collections[0], ax=ax, orientation='vertical')
cbar.set_label('decoding accuracy above null', rotation=90, labelpad=15)

ax.set_title(f'{sel_structure} {sel_unit_subsample_size} units; coronal view')



In [ ]:
#plot ccf_dv vs. ccf_ap vs. decoding accuracy

fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.imshow(np.max(area_mask[:,:,:],axis=2).T,cmap='gray',alpha=0.5)
# ax.imshow(np.max(combined_area_mask[:,:,:],axis=2).T,cmap='magma',alpha=0.5)

ax.scatter((plot_data['ccf_ap']/10),plot_data['ccf_dv']/10,c=plot_data['mean_diff'],s=20,cmap='Reds',vmin=0,vmax=0.2)

ax.set_ylabel('CCF DV')
ax.set_xlabel('CCF AP')

#set xlim and ylim to be around the area mask
ax.set_xlim([np.nanmin(plot_data['ccf_ap']/10)-100,np.nanmax(plot_data['ccf_ap']/10)+100])
ax.set_ylim([np.nanmin(plot_data['ccf_dv']/10)-100,np.nanmax(plot_data['ccf_dv']/10)+100])

ax.invert_yaxis()

#make colorbar
cbar = plt.colorbar(ax.collections[0], ax=ax, orientation='vertical')
cbar.set_label('decoding accuracy above null', rotation=90, labelpad=15)

ax.set_title(f'{sel_structure} {sel_unit_subsample_size} units; sagittal view')



## Plot decoder accuracy versus session-wise behavioral performance
### (should include bad behavior sessions as long as mouse was engaged)

In [ ]:
if type(results_session_pd['cross_modality_dprime_aud_blocks'].iloc[0])==str:
    print("column is string, needs to be converted to list of floats")

    mean_cross_modal_dprime_vis_blocks=[]
    mean_cross_modal_dprime_aud_blocks=[]

    for rr, row in results_session_pd.iterrows():
        if row['cross_modality_dprime_aud_blocks'] is not None:
            temp_list=row['cross_modality_dprime_aud_blocks'].split(';')
            new_list=[]
            for ll in temp_list:
                new_list.append(float(ll))
            mean_cross_modal_dprime_aud_blocks.append(np.nanmean(new_list))
        else:
            mean_cross_modal_dprime_aud_blocks.append(np.nan)
        
        if row['cross_modality_dprime_vis_blocks'] is not None:
            temp_list=row['cross_modality_dprime_vis_blocks'].split(';')
            new_list=[]
            for ll in temp_list:
                new_list.append(float(ll))
            mean_cross_modal_dprime_vis_blocks.append(np.nanmean(new_list))
        else:
            mean_cross_modal_dprime_vis_blocks.append(np.nan)

    results_session_pd['mean_cross_modality_dprime_vis_blocks']=mean_cross_modal_dprime_vis_blocks
    results_session_pd['mean_cross_modality_dprime_aud_blocks']=mean_cross_modal_dprime_aud_blocks

else:
    print("column is already list of floats, just need to take mean")

    results_session_pd['mean_cross_modality_dprime_vis_blocks']=results_session_pd['cross_modality_dprime_vis_blocks'].apply(lambda x: np.nanmean(x) if x is not None else np.nan)
    results_session_pd['mean_cross_modality_dprime_aud_blocks']=results_session_pd['cross_modality_dprime_aud_blocks'].apply(lambda x: np.nanmean(x) if x is not None else np.nan)

results_session_pd['mean_cross_modality_dprime']=results_session_pd[['mean_cross_modality_dprime_vis_blocks','mean_cross_modality_dprime_aud_blocks']].mean(axis=1)

In [ ]:
# correlate session-wise accuracy with cross-modal dprime

sel_structure='MOs'
sel_unit_subsample_size=10

fig,ax=plt.subplots(1,1,figsize=(5,4))

plot_data=results_session_pd.query('unit_subsample_size==@sel_unit_subsample_size and structure==@sel_structure')

r,p=stats.pearsonr(plot_data[['mean_cross_modality_dprime_aud_blocks',
                      'mean_cross_modality_dprime_vis_blocks']].mean(axis=1),
                    plot_data['mean_diff'])

ax.scatter(plot_data[['mean_cross_modality_dprime_aud_blocks',
                      'mean_cross_modality_dprime_vis_blocks']].mean(axis=1),
            plot_data['mean_diff'],marker='.',color='k',
            label='true')

ax.set_ylabel('decoding accuracy above null')
ax.set_xlabel('mean cross-modal dprime')
ax.set_title(sel_structure+f' (r={r:.2f}, p={p:.3e})')

fig.tight_layout()


In [ ]:
# compare correlation across areas

decoding_accuracy_vs_dprime_by_structure={
    'structure':[],
    'unit_subsample_size':[],
    'r':[],
    'p':[],
}

for sel_structure in results_session_pd['structure'].unique():
    for sel_unit_subsample_size in results_session_pd['unit_subsample_size'].unique():

        if np.isnan(sel_unit_subsample_size):
            structure_data=results_session_pd.query('unit_subsample_size.isna() and structure==@sel_structure')
        else:
            structure_data=results_session_pd.query('unit_subsample_size==@sel_unit_subsample_size and structure==@sel_structure')

        if len(structure_data['mean_diff'])<10:
            continue

        r,p=stats.pearsonr(structure_data[['mean_cross_modality_dprime_aud_blocks','mean_cross_modality_dprime_vis_blocks']].mean(axis=1),
                            structure_data['mean_diff'])
        
        decoding_accuracy_vs_dprime_by_structure['structure'].append(sel_structure)
        if np.isnan(sel_unit_subsample_size):
            decoding_accuracy_vs_dprime_by_structure['unit_subsample_size'].append(np.nan)
        else:
            decoding_accuracy_vs_dprime_by_structure['unit_subsample_size'].append(sel_unit_subsample_size)
        decoding_accuracy_vs_dprime_by_structure['r'].append(r)
        decoding_accuracy_vs_dprime_by_structure['p'].append(p)

decoding_accuracy_vs_dprime_by_structure_df=pd.DataFrame(decoding_accuracy_vs_dprime_by_structure)


In [ ]:
results_session_pd['unit_subsample_size'].unique()

In [ ]:
sel_unit_subsample_size=10

fig,ax=plt.subplots(1,1,figsize=(12,4))

decoding_accuracy_vs_dprime_by_structure_df.query('unit_subsample_size==@sel_unit_subsample_size').sort_values(by='r',ascending=False).plot.bar(ax=ax,x='structure',y=['r'],rot=90)
ax.set_ylabel('correlation coefficient')
ax.set_title('correlation between decoding accuracy and cross-modal dprime')

fig.tight_layout()


In [ ]:
sel_structure='MRN'

plot_data=decoding_accuracy_vs_dprime_by_structure_df.query('structure==@sel_structure').sort_values(by='unit_subsample_size',ascending=True)[['unit_subsample_size','r']]
xvect=np.arange(0,plot_data.shape[0])

fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.plot(xvect,plot_data['r'],marker='o',color='k')
ax.set_xticks(xvect)
ax.set_xticklabels(plot_data['unit_subsample_size'].values)

ax.set_xlabel('unit subsample size')
ax.set_ylabel('correlation coefficient')
ax.set_title(sel_structure)

## Supplemental plots

In [ ]:
# supplemental: plot shifts, null, etc. for a few example sessions

# also average across sessions


In [ ]:
# "all units" accuracy versus total_n_units

sel_structure='MOs'

fig,ax=plt.subplots(1,1,figsize=(5,4))

plot_data=results_session_pd.query('unit_subsample_size.isna() and structure==@sel_structure')
ax.scatter(plot_data['total_n_units'],
            plot_data['mean_true'],marker='.',color='k',
            label='true')

ax.scatter(plot_data['total_n_units'],
           plot_data['median_null'],marker='.',color='gray',alpha=0.5,
           label='null')

ax.set_ylabel('decoding accuracy')
ax.set_xlabel('total n units')
ax.set_title(sel_structure)
ax.legend()

In [ ]:
# sel_session='742903_2024-10-22'
# # sel_session='715710_2024-07-15'
# # sel_session='670180_2023-07-27'

# test_session_df=(
#     pl.scan_parquet(results_path)
#     .with_columns(
#         pl.col('electrode_group_names').flatten().n_unique().eq(1).over(grouping_cols - {'electrode_group_names'}).alias('is_sole_recording'),
#     )
#     .filter(
#         pl.col('session_id').eq(sel_session),
#         combine_multi_probe_expr,
#         pl.col('is_all_trials').not_(),
#     )
# ).collect()
# test_session_df.to_pandas()['structure'].unique()

### Single example session - show linear shift, repeats, etc.

In [ ]:
# get all shifts for an example session
sel_session='742903_2024-10-22'
# sel_session='715710_2024-07-15'
# sel_session='670180_2023-07-27' #templeton

example_session_df=decoding_utils.load_single_session_decoder_accuracy(results_path, sel_session, combine_multi_probe_rec=True)

In [ ]:
example_session_df['structure'].unique()

In [ ]:
sel_structure='MOs'
sel_n_units=10
example_session_area=example_session_df.to_pandas().query('unit_subsample_size==@sel_n_units and structure==@sel_structure')

mean_across_repeats=np.nanmean(np.vstack(example_session_area['balanced_accuracy_test'].values),axis=0)

# all_shifts_aligned=np.full((25,200),dtype=float,fill_value=np.nan)

fig,ax=plt.subplots(1,1,figsize=(5,4))
for rr, row in example_session_area.iterrows():
    ax.plot(row['shift_idx'],row['balanced_accuracy_test'],marker='',color='k',alpha=0.5,linewidth=0.5)
ax.set_xlabel('shift index')
ax.set_ylabel('decoding accuracy')
ax.set_title(f'{sel_session} - {sel_structure}')

#take the mean of all repeats
ax.plot(row['shift_idx'],mean_across_repeats,color='k',label='mean across repeats',linewidth=2)

fig.tight_layout()

aligned_accuracy = mean_across_repeats[row['shift_idx']==0][0]
median_null_accuracy = np.median(mean_across_repeats[row['shift_idx']!=0])

# plot the true accuracy versus the histogram of the null accuracy
fig,ax= plt.subplots(1,1,figsize=(5,4))
xbins=np.linspace(0, 1, 50)
ax.hist(mean_across_repeats[row['shift_idx']!=0], bins=xbins, alpha=0.5, label='null accuracy')
ax.axvline(aligned_accuracy, color='r', linestyle='--', label='true accuracy')
ax.axvline(median_null_accuracy, color='k', linestyle='--', label='median null accuracy')
ax.set_xlabel('decoding accuracy')
ax.set_ylabel('n shifts')
ax.set_title(f'{sel_session} - {sel_structure} \n true accuracy: {aligned_accuracy:.3f}, median null accuracy: {median_null_accuracy:.3f}')
ax.legend()
fig.tight_layout()

#get the full histogram of all shift-repeats
all_aligned_accuracy= np.vstack(example_session_area['balanced_accuracy_test'].values)[:,row['shift_idx']==0].flatten()
all_null_accuracy = np.vstack(example_session_area['balanced_accuracy_test'].values)[:,row['shift_idx']!=0].flatten()

all_aligned_accuracy_median= np.median(all_aligned_accuracy)
all_null_accuracy_median= np.median(all_null_accuracy)

fig,ax= plt.subplots(1,1,figsize=(5,4))
xbins=np.linspace(0, 100, 50)
ax.hist(all_null_accuracy*100, bins=xbins, alpha=0.5, label='null accuracy',density=True)
ax.hist(all_aligned_accuracy*100, bins=xbins, alpha=0.5, label='aligned accuracy', density=True)
ax.axvline(all_aligned_accuracy_median*100, color='r', linestyle='--', label='aligned accuracy median')
ax.axvline(all_null_accuracy_median*100, color='k', linestyle='--', label='null accuracy median')
ax.set_xlabel('decoding accuracy')
ax.set_ylabel('density')
ax.set_title(f'{sel_session} - {sel_structure} \n true accuracy: {all_aligned_accuracy_median:.3f}, median null accuracy: {all_null_accuracy_median:.3f}')
ax.legend()
fig.tight_layout()


In [ ]:
# results_session_pd.query('session_id=="742903_2024-10-22" and structure=="ACAd"')

In [ ]:
# accuracy above null vs. true accuracy

fig,ax=plt.subplots(1,1,figsize=(5,4))

r,p=stats.pearsonr(results_session_pd['mean_true'],
                    results_session_pd['mean_diff'])

ax.plot(results_session_pd['mean_true'],results_session_pd['mean_diff'],'k.',alpha=0.2)
ax.set_xlabel('mean true accuracy')
ax.set_ylabel('decoding accuracy above null')
ax.set_title(f'r: {r:.3f}, p: {p:.3e}')

fig.tight_layout()

In [ ]:
#calculate linear shift pvalue
all_mean_true=results_session_pd['mean_true'].values
all_balanced_accuracy_test=results_session_pd['balanced_accuracy_test'].values
all_percentile=[]

for tt, true_value in enumerate(all_mean_true):
    shift_values=all_balanced_accuracy_test[tt][all_balanced_accuracy_test[tt]!=true_value]
    percentile=stats.percentileofscore(shift_values,true_value)
    all_percentile.append(percentile)

results_session_pd['percentile']=all_percentile
results_session_pd['p_value']=1-(results_session_pd['percentile']/100)


In [ ]:
# linear shift pvalue vs. true accuracy
fig,ax=plt.subplots(1,1,figsize=(5,4))

r,p=stats.pearsonr(results_session_pd['mean_true'],
                    results_session_pd['p_value'])

ax.plot(results_session_pd['mean_true'],results_session_pd['p_value'],'k.',alpha=0.2)
ax.set_yscale('log')
ax.set_xlabel('mean true accuracy')
ax.set_ylabel('linear shift p-value')
ax.set_title(f'r: {r:.3f}, p: {p:.3e}')

In [ ]:
#linear shift pvalue vs. diff from null
fig,ax=plt.subplots(1,1,figsize=(5,4))

r,p=stats.pearsonr(results_session_pd['mean_diff'],
                    results_session_pd['p_value'])

ax.plot(results_session_pd['mean_diff'],results_session_pd['p_value'],'k.',alpha=0.2)
ax.set_yscale('log')
ax.set_xlabel('mean accuracy above null')
ax.set_ylabel('linear shift p-value')
ax.set_title(f'r: {r:.3f}, p: {p:.3e}')

In [ ]:
not_nan_vals=~np.isnan(np.log10(results_session_pd['mean_true'].values)) & ~np.isnan(np.log10(results_session_pd['p_value'].values))
stats.pearsonr(results_session_pd['mean_true'].values[not_nan_vals],np.log10(results_session_pd['p_value'].values[not_nan_vals]))

In [ ]:
# get all shifts for an example area across sessions

sel_structure='ACAd'
sel_unit_subsample_size=10

fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)

structure_results=results_session_pd.query('structure==@sel_structure and unit_subsample_size==@sel_unit_subsample_size and n_passing_blocks>=4')
for rr, row in structure_results.iterrows():
    ax[0].plot(row['shift_idx'],row['balanced_accuracy_test'],marker='',color='k',alpha=0.5,linewidth=0.5)

ax[0].set_xlabel('shift index')
ax[0].set_ylabel('decoding accuracy')
ax[0].set_title(f'good behavior ({len(structure_results)} sessions)')

structure_results=results_session_pd.query('structure==@sel_structure and unit_subsample_size==@sel_unit_subsample_size and n_passing_blocks<4')
for rr, row in structure_results.iterrows():
    ax[1].plot(row['shift_idx'],row['balanced_accuracy_test'],marker='',color='k',alpha=0.5,linewidth=0.5)

ax[1].set_xlabel('shift index')
ax[1].set_ylabel('decoding accuracy')
ax[1].set_title(f'bad behavior ({len(structure_results)} sessions)')

fig.suptitle(f'{sel_structure} - {sel_unit_subsample_size} unit subsample')